In [2]:
import nest_asyncio
nest_asyncio.apply()

In [5]:

from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)




@dataclass
class UserContext:
    text: str
    translated_text: str = ""    
    
    
def translate_instructions(wrappers,agent):
    return (
        "You are a helpful assistant. Translate this text into Urdu:\n\n"
        f"\"{wrappers.context.text}\""
    )
 
 
def story_instructions(wrapper, agent):
    return (
        "You are a creative story writer. Write a short story based on the Urdu text:\n\n"
        f"\"{wrapper.context.translated_text}\""
    )

async def main(user_input: str): 
    ctx = UserContext(text=user_input) 
    
    translator = Agent[UserContext](
        name="Urdu Translator",
        instructions=translate_instructions,
        model=model
    ) 
    
    trans_result = await Runner.run(
        starting_agent=translator,
        input=user_input,
        context=ctx
    ) 
    ctx.translated_text = trans_result.final_output.strip() 
    story_agent = Agent[UserContext](
        name="Story Generator",
        instructions=story_instructions,
        model=model
    ) 
 
    story_result = await Runner.run(
        starting_agent=story_agent,
        input=ctx.translated_text,
        context=ctx
    )
 
    print(f"original text : {ctx.text}")
    print("→ Urdu Translation:\n", ctx.translated_text)
    print("\n→ Generated Story:\n", story_result.final_output)
if __name__ == "__main__":
    
    text  = "allama iqbal is our national poet"

    asyncio.run(main(text ))

original text : allama iqbal is our national poet
→ Urdu Translation:
 Here's the translation of "Allama Iqbal is our national poet" into Urdu:

**علامہ اقبال ہمارے قومی شاعر ہیں۔**

**(Allama Iqbal hamare qaumi shayar hain.)**

→ Generated Story:
 The dust motes danced in the single ray of sunlight slicing through the classroom. Outside, the air throbbed with the promise of mango season, a sweet, drowsy hum against the backdrop of Karachi’s relentless traffic. Inside, however, ten-year-old Zara struggled to focus. The Urdu words blurred on the chalkboard: **علامہ اقبال ہمارے قومی شاعر ہیں۔** *(Allama Iqbal hamare qaumi shayar hain.)* Allama Iqbal is our national poet.

For weeks, these words had echoed in her head. Her teacher, Miss Fatima, stressed the importance of Iqbal's poetry, his philosophy, his role in inspiring the dream of Pakistan. Zara understood, intellectually, the significance. But she longed to *feel* it.

"Zara, beta (child)," Miss Fatima's gentle voice pulled her bac